In [1]:
import pandas as pd
import re
import time

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, explode, from_json, col, current_date,current_timestamp, lit, to_date, month

from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, StructType, StructField, ArrayType


from pyspark.sql import SparkSession
import pyspark

In [2]:
CATALOG_URI = "http://nessie:19120/api/v1"
WAREHOUSE = "s3a://gold/"              
STORAGE_URI = "http://minio:9000"
AWS_ACCESS_KEY = "admin"
AWS_SECRET_KEY = "password"

In [3]:
conf = (
    pyspark.SparkConf()
        .setAppName('silver_transform')

        # 📦 Dependencias necesarias
        .set("spark.jars.packages", ",".join([
            "org.postgresql:postgresql:42.7.3",
            "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0",
            "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.77.1",
            "software.amazon.awssdk:bundle:2.24.8",
            "software.amazon.awssdk:url-connection-client:2.24.8",
            "org.apache.hadoop:hadoop-aws:3.3.4"
        ]))

        # 🧩 Extensiones Iceberg + Nessie
        .set("spark.sql.extensions", ",".join([
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
            "org.projectnessie.spark.extensions.NessieSparkSessionExtensions"
        ]))

        # 🗂️ Catálogo Nessie
        .set("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
        .set("spark.sql.catalog.nessie.uri", CATALOG_URI)
        .set("spark.sql.catalog.nessie.ref", "main")
        .set("spark.sql.catalog.nessie.authentication.type", "NONE")
        .set("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")
        .set("spark.sql.catalog.nessie.warehouse", WAREHOUSE)

        # ☁️ Configuración S3A para MinIO
        .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .set("spark.hadoop.fs.s3a.endpoint", STORAGE_URI)
        .set("spark.hadoop.fs.s3a.path.style.access", "true")
        .set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY)
        .set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_KEY)
        .set("spark.hadoop.fs.s3a.aws.credentials.provider",
             "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
        .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

        # ⚡ Optimizaciones de ejecución
        .set("spark.sql.execution.arrow.pyspark.enabled", "true")
        .set("spark.sql.parquet.filterPushdown", "true")
        .set("spark.sql.parquet.mergeSchema", "false")
        .set("spark.sql.shuffle.partitions", "64")  # 🔧 más particiones para distribuir carga
        .set("spark.sql.files.maxPartitionBytes", "64MB")  # ⚖️ reduce tamaño de tarea para evitar saturación

        .set("spark.driver.memory", "5g")                     # Driver usa hasta 5 GB (de los 6g disponibles)
        .set("spark.executor.memory", "6g")                   # Cada executor usa hasta 6 GB (de los 8g disponibles)
        .set("spark.executor.cores", "4")                     # Más núcleos por executor para paralelismo
        .set("spark.driver.maxResultSize", "2g")              # Aumenta el límite de resultados del driver
        .set("spark.network.timeout", "600s")                 # Timeout más generoso para cargas grandes
        .set("spark.executor.heartbeatInterval", "60s")       # Latido coherente con el timeout
        
        # ⚙️ Escritura
        .set("spark.sql.parquet.compression.codec", "snappy")

)

In [4]:
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()
print("Spark Session Started")

Spark Session Started


In [ ]:
posts_df = spark.read.table("nessie.silver.posts")
votes_df = spark.read.table("nessie.silver.votes")